In [47]:
import pandas as pd
import tqdm
import numpy as np

In [2]:
nfl_schools = pd.read_csv('unclean_data/university_info/nfl_player_unis.csv')

In [3]:
nfl_stats = pd.read_csv('data/nfl_av_ratings_all.csv')

In [4]:
nfl_stats.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Rk', 'Player', 'Year', 'Age', 'Draft',
       'Tm', 'Lg', 'G', 'GS', 'Yrs', 'PB', 'AP1', 'AV', 'percentile_av'],
      dtype='object')

In [5]:
nfl_stats.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)

In [6]:
nfl_stats.head()

,Rk,Player,Year,Age,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV,percentile_av
0,1,Aaron Rodgers\RodgAa00,2011,28.0,1-24,GNB,NFL,15.0,15.0,1,1,1,23.0,100
1,2,Tom Brady\BradTo00,2011,34.0,6-199,NWE,NFL,16.0,16.0,1,1,0,21.0,99
2,3,Drew Brees\BreeDr00,2011,32.0,2-32,NOR,NFL,16.0,16.0,1,1,0,20.0,99
3,4,Patrick Peterson\PetePa00,2011,21.0,1-5,ARI,NFL,16.0,16.0,1,1,1,20.0,99
4,5,Jahri Evans\EvanJa21,2011,28.0,4-108,NOR,NFL,16.0,16.0,1,1,1,19.0,99


In [7]:
nfl_stats['name'] = ''
len(nfl_stats.columns)

15

In [22]:
for i in range(len(nfl_stats)):
    nfl_stats.iat[i, 1] = nfl_stats.iat[i, 1].split('\\')[0]

In [23]:
nfl_stats.head()

,Rk,Player,Year,Age,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV,percentile_av,name
0,1,Aaron Rodgers,2011,28.0,1-24,GNB,NFL,15.0,15.0,1,1,1,23.0,100,
1,2,Tom Brady,2011,34.0,6-199,NWE,NFL,16.0,16.0,1,1,0,21.0,99,
2,3,Drew Brees,2011,32.0,2-32,NOR,NFL,16.0,16.0,1,1,0,20.0,99,
3,4,Patrick Peterson,2011,21.0,1-5,ARI,NFL,16.0,16.0,1,1,1,20.0,99,
4,5,Jahri Evans,2011,28.0,4-108,NOR,NFL,16.0,16.0,1,1,1,19.0,99,


In [8]:
nfl_schools.drop(columns = ['Unnamed: 0'], inplace = True)

In [37]:
nfl_schools.columns

Index(['school_name', 'college', 'Player', 'state'], dtype='object')

In [27]:
nfl_schools.rename(columns = {'index': 'school_name', 'name': 'Player'}, inplace = True)

In [38]:
for i in range(len(nfl_schools)):
    nfl_schools.iat[i, 2] = nfl_schools.iat[i, 2].strip()

In [24]:
df = nfl_stats[['Player', 'Year', 'Tm', 'AV', 'percentile_av']]

In [36]:
x = 'Aaron Rodgers '
x.strip()

'Aaron Rodgers'

In [42]:
merged = df.merge(nfl_schools, how = 'left')

In [53]:
merged.loc[merged['Player'] == 'Patrick Peterson'].iat[0, 7]

nan

In [58]:
merged.drop(columns = ['school_name'], inplace = True)

In [64]:
merged.dropna(thresh = 6, inplace = True)

In [80]:
merged.state.unique()

array(['CA', 'MI', 'IN', nan, 'PA', 'AL', 'SC', 'NE', 'OR', 'MO', 'TX',
       'NJ', 'GA', 'TN', 'MS', 'WA', 'AZ', 'FL', 'UT', 'MD', 'MA', 'AR',
       'CO', 'NC', 'IA', 'NY', 'NV', 'WI', 'IL', 'WV', 'KY', 'LA', 'VA',
       'WY', 'OK', 'OH', 'ID', 'HI', 'ME', 'NM', 'MN', 'ND', 'KS', 'CT',
       'NH', 'RI', 'SD'], dtype=object)

In [71]:
college_list = merged[merged.isnull().any(axis=1)].college.unique().tolist()

In [73]:
len(college_list)

179

In [74]:
list_df = pd.DataFrame(columns = ['college', 'st'], index = [i for i in range(179)])

In [75]:
list_df['college'] = college_list

In [78]:
list_df.to_csv('college_st.csv')

In [81]:
list_df = pd.read_csv('college_st.csv')

In [90]:
# list_df.drop(columns = ['Unnamed: 0'], inplace = True)
list_df.rename(columns= {'st':'state'}, inplace = True)

In [105]:
merged = merged.merge(list_df, how = 'left', on = 'college')
merged.head()

,Player,Year,Tm,AV,percentile_av,college,state_x,state_y,state
0,Aaron Rodgers,2011,GNB,23.0,100,California,CA,NaN,NaN
1,Tom Brady,2011,NWE,21.0,99,Michigan,MI,NaN,NaN
2,Drew Brees,2011,NOR,20.0,99,Purdue,IN,NaN,NaN
3,Patrick Peterson,2011,ARI,20.0,99,LSU,NaN,LA,LA
4,Jahri Evans,2011,NOR,19.0,99,Bloomsburg,PA,NaN,NaN


In [106]:
for i in range(len(merged)):
    if pd.isnull(merged.iat[i, 6]):
        merged.iat[i, 6] = merged.iat[i, 7]

In [114]:
merged.drop(columns = ['state_y', 'state', 'college'], inplace = True)

In [112]:
merged.rename(columns = {'state_x':'state'}, inplace = True)

In [116]:
merged.to_csv('data/nfl_player_av_state.csv')

In [136]:
nba_stats = pd.read_csv('data/nba_per_ratings_all.csv').drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])[['Player', 'Season', 'PER', 'percentile_per']]
for i in range(len(nba_stats)):
    nba_stats.iat[i, 0] = nba_stats.iat[i, 0].split('\\')[0]
nba_stats.head()

,Player,Season,PER,percentile_per
0,Orien Greene,2010-11,33.0,99
1,LeBron James,2010-11,27.3,98
2,Dwight Howard,2010-11,26.1,98
3,Dwyane Wade,2010-11,25.6,98
4,Kevin Love,2010-11,24.3,97


In [144]:
nba_schools = pd.read_csv('unclean_data/university_info/nba_player_unis.csv').drop(columns = 'Unnamed: 0').rename(columns = {'index': 'school'})

In [192]:
nba_schools.head()

,school,Player,state
0,Acadia University,Brian Heaney,NaN
1,Alabama A&M University,Mickell Gladness,AL
2,Alabama State University,Kevin Loder,AL
3,Albany State University,Caldwell Jones,GA
4,Albany State University,Major Jones,GA


In [169]:
x = pd.DataFrame(columns = {'school', 'state'}, index = [i for i in range(105)])
x['school'] = nba_schools[nba_schools['state'].isnull()].school.unique()

In [171]:
x.to_csv('nba_unknown_schools.csv')

In [176]:
new_schools = pd.read_csv('nba_unknown_schools.csv').drop(columns = ['Unnamed: 0'])

In [187]:
merged = nba_schools.merge(new_schools, how = 'left', on = 'school')
for i in range(len(merged)):
    if pd.isnull(merged.iat[i, 2]):
        merged.iat[i, 2] = merged.iat[i, 3]
merged.drop(columns = ['state_y', 'school'], inplace = True)
merged.rename(columns = {'state_x': 'state'}, inplace = True)
merged.head()

,Player,state
0,Brian Heaney,PA
1,Mickell Gladness,AL
2,Kevin Loder,AL
3,Caldwell Jones,GA
4,Major Jones,GA


In [197]:
merged = nba_stats.merge(merged, how = 'left', on = 'Player').dropna(how = 'any').sort_values(by = 'PER', ascending = False)

In [202]:
merged.drop(1398, inplace = True)

In [208]:
merged.to_csv('data/nba_player_per_state.csv', index = False)

In [239]:
mlb_stats = pd.read_csv('data/mlb_war_ratings_all.csv')[['Player', 'WAR/pos', 'Year', 'Tm', 'percentile_per']]
mlb_stats['player_id'] = ''
for i in range(len(mlb_stats)):
    name, pid = mlb_stats.iat[i, 0].split('\\')
    mlb_stats.iat[i, 0] = name
    mlb_stats.iat[i, 5] = pid

In [241]:
mlb_stats.head()

,Player,WAR/pos,Year,Tm,percentile_per,player_id
0,Jose Bautista,99.882629,2011,TOR,100,bautijo02
1,Jacoby Ellsbury,99.877526,2011,BOS,99,ellsbja01
2,Matt Kemp,99.831598,2011,LAD,99,kempma01
3,Dustin Pedroia,99.826495,2011,BOS,99,pedrodu01
4,Ryan Braun,99.739743,2011,MIL,99,braunry02


In [270]:
mlb_schools = pd.read_csv('unclean_data/university_info/mlb_player_unis.csv')[['state', 'playerID']]

In [272]:
mlb_schools.head()

,state,playerID
0,PA,aardsda01
1,TX,aardsda01
2,GA,abadan01
3,VT,abbeybe01
4,KY,abbotje01


In [273]:
mlb_schools.rename(columns = {'playerID':'player_id'}, inplace = True)

In [274]:
mlb_schools.loc[mlb_schools['player_id'] == 'pedrodu01']

,state,player_id
5171,AZ,pedrodu01


In [260]:
for i in range(len(mlb_schools)):
    mlb_schools.iat[i, 2] = mlb_schools.iat[i, 2].strip()
for i in range(len(mlb_stats)):
    mlb_stats.iat[i, 5] = mlb_stats.iat[i, 5].strip()

In [275]:
merged = mlb_stats.merge(mlb_schools, how = 'left')

In [278]:
merged.dropna(how = 'any', inplace = True)

In [279]:
merged.to_csv('data/mlb_war_player_state.csv')